# Aula 06 &mdash; Data Scraping com API do Twitter

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

In [1]:
import pandas as pd
import numpy as np

import tweepy

from functools import reduce
from collections import Counter
import json

## Introdução

Nas aulas anteriores vimos como obter dados diretamente de uma página web. Para isso, utilizamos a biblioteca BeautifulSoup. A biblioteca facilitava a busca de tags e, consequentemente, de informações úteis nas páginas. Embora tenhamos que lançar mão desse recurso em muitas ocasiões, alguns sites fornecem APIs para obtermos dados de forma semi-estruturada. Esses dados possuem formato livre, mas possuem indicadores do conteúdo. As representações mais usuais são [XML](https://en.wikipedia.org/wiki/XML) e [JSON](https://en.wikipedia.org/wiki/JSON).

O formato JSON tem se tornado um dos mais usados para troca de dados na internet. Ele tem substituido inclusive XML. Muitos bancos de dados NoSQL, como MongoDB, fornecem suporte direto a dados nesse formato. Em outras palavras, JSON é, talvez, o formato franco de representação de dados para Data Science. Sendo assim, discutiremos rapidamente sua representação.

## JSON

O formato JSON (lê-se como *jay-son*) é essencialmente um dicionário. Ele descreve uma sequência de pares (chave,valor). Objetos no formato JSON são delimitados por { e }. Por sua vez, os pares são separados por vírgulas. As chaves e valores são separados por : (dois-pontos).

As chaves devem ser sempre strings em unicode. Os valores podem ser:

- string
- número
- objeto
- vetor
- true
- false
- null

Os vetores referenciados acima são como vetores/listas em uma linguagem de programação. Eles podem conter uma sequência de um ou mais valores, incluindo outros vetores e objetos. A gramática do formato pode ser conferida em http://www.json.org/json-pt.html.

### Exemplo de um dado no formato JSON

O exemplo a seguir foi retirado da Wikipedia. Ele ilustra como poderiam ser armazenados diversos dados sobre uma pessoa. A escolha das chaves é completamente arbitrária e definida pelo desenvolvedor.

```json
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 25,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    },
    {
      "type": "mobile",
      "number": "123 456-7890"
    }
  ],
  "children": [],
  "spouse": null
}
```

## API do Twitter

O Twitter oferece aos desenvolvedores uma API para permitir a interação entre o site e aplicativos de terceiros. A API é uma das ricas entre as redes sociais e, por isso, muitas pesquisas comerciais e científicas usam a plataforma.

A API do Twitter se subdivide em outras duas APIs: REST e Streaming. A API de REST permite aos desenvolvedores acessar os perfis dos usuários (públicos) assim como seus seguidores, listas, timeline, além de efetuar buscas por termos ou usuários na plataforma. A API de Streaming, por outro lado, é dedicada ao monitoramento em tempo real dos tweets sendo postados na plataforma.

Além dessas duas interfaces públicas, o Twitter ainda oferece uma terceira interface comercial, Gnip, para acesso a grandes volumes de dados. O Gnip, basicamente, fornece acesso à base de dados (pública) inteira do Twitter. Enquanto a API de Streaming faz amostragem dos tweets em tempo real (não é muito claro como isso é feito), o Gnip fornece acesso a efetivamente todos as mensagens sendo postadas. O mesmo acontece em relação ao REST, que limita, por exemplo, a busca por termos, não retornando todos os resultados. O Gnip, ao contrário, permite efetuar uma busca em toda a base.

## Acesso à API do Twitter

O acesso à API requer a criação de um perfil na plataforma e registro de um aplicativo. A autenticação é feita através do protocolo [OAuth](https://en.wikipedia.org/wiki/OAuth). A ideia básica do protocolo é que um usuário possa delegar acesso a recursos em um determinado site a terceiros sem revelar suas credenciais. Resumindo, o protocolo permite que chaves de acesso sejam geradas pelo usuário; essas chaves são passadas a um terceiro, a quem se deseja permitir acesso aos recursos do usuário; finalmente, a parte a quem o acesso foi delegado pode conectar-se ao sistema usando as chaves que lhe foram enviadas.

#### Registrando um novo aplicativo

Para registrar um aplicativo e gerar novas chaves de acesso, o usuário deve ir até o site https://apps.twitter.com.

![https://apps.twitter.com](images/tela_inicial_apps_twitter.png)

Após logar, a tela inicial mostrará as aplicações autorizadas no momento.

![https://apps.twitter.com](images/apps_twitter_logado.png)

Para criar/autorizar um novo aplicativo, clique no botão **Create New App**. Em seguida, você será redirecionado para uma página onde deverá preencher os dados do aplicativo

![https://apps.twitter.com](images/create_new_app.png)

e aceitar os termos e condições de uso

![https://apps.twitter.com](images/create_new_app_accept_terms_confirm.png)

Pronto! Agora você já resgistrou o aplicativo e poderá gerar as chaves de acesso como a seguir.

#### Obtendo as chaves de acesso

Após registrar o novo aplicativo, você será redirecionado à página de gerenciamento do aplicativo

![https://apps.twitter.com](images/new_app_twitter.png)

Configure as permissões da maneira como for mais conveniente. Em geral, para apenas minerar os dados, basta permitir que o aplicativo possa somente ler os dados. Isso evita, ou diminui as chances, de postagens indevidas caso alguém obtenha suas chaves.

Para gerar as chaves de acesso, vá até a aba **Keys and Access Tokens**.

![https://apps.twitter.com](images/consumer_key_twitter.png)

Inicialmente o aplicativo não está autorizado a logar em nome do usuário. Você verá que foram assinalados um identificador e uma chave de acesso de usuário (*Consumer key and secret*), como mostrado na figura acima. Essas chaves servem para autenticar o usuário, contudo, elas não autenticam o aplicativo, como mostrado na figura abaixo.

![https://apps.twitter.com](images/authorize_app_twitter.png)

Para autorizar o aplicativo, clique no botão **Create my access tokens**. O Twitter então gerará novos identificadores e chaves de acesso para o aplicativo (*Access token and secret*). Esses deverão ser usados em conjunto com as chaves de usuário para se ter acesso à API.

![https://apps.twitter.com](images/access_tokens_twitter.png)

É importante lembrar que esses dois pares de identificadores e chaves nunca devem estar legíveis em seu programa. Eles são como senhas e, portanto, devem ser protegidos.

Agora que já obtivemos as chaves de acesso, podemos iniciar a coleta de dados.

## Acessando a API com Tweepy

Existem diversas bibliotecas de Python para acessar a API do Twitter, como listado na página dos desenvolvedores (https://dev.twitter.com/resources/twitter-libraries). As duas bibliotecas mais usadas (pelo que vi na internet) são [Python-Twitter](https://github.com/bear/python-twitter) e [Tweepy](https://github.com/tweepy/tweepy). No entanto, Tweepy está melhor documentada e possui melhor suporte à API de Streaming. Assim, usaremos Tweepy em nossos exemplos.

#### Conectando-se a API com Tweepy

A biblioteca Tweepy, na verdade, é um wrapper para os serviços listados na API do Twitter. Ela se encarrega dos detalhes de codificação e requisições ao servidor, abstraindo as consultas através de funções e métodos. A biblioteca abstrai inclusive os detalhes de autenticação.

O primeiro passo para coletar os dados é autenticar o usuário e o aplicativo. Isso é feito da seguinte forma:


In [2]:
# As chaves estao armazenadas em um arquivo de configuracoes
# Python possui uma biblioteca para processar esses arquivos (ver configparser)
with open('../configs.txt') as f:
    chaves = json.load(f)
    
# Autentica o usuario na API
auth = tweepy.OAuthHandler(chaves["consumer_key"], chaves["consumer_secret"])

# Autentica o aplicativo
auth.set_access_token(chaves["access_token"], chaves["access_token_secret"])


O objeto `auth` contém a sessão autenticada e deverá ser usado nas conexões com a API.

#### Acesso à REST API

O acesso à REST API é feito através de objetos da classe `tweepy.API`. Os objetos dessa classe possuem métodos que encapsulam os acessos às funcionalidades da API. Assim, devemos instaciar a classe usando a conexão criada anteriormente. É importante respeitar os limites impostos pela API, caso contrário, podemos ser bloqueados. As requisições que ultrapassarem os limites do Twitter resultaram em exceções do tipo `tweepy.RateLimitError`. Tweepy pode controlar automaticamente esses acessos. Podemos estabelecer que as chamadas a determinados métodos devem esperar até que o limite de requisições seja reestabelecido. O limite de requisições varia dependendo do tipo de requisição. O Twitter também faz constantes modificações nesses limites. Então, você sempre deve consultar a documentação da API para se certificar dos limites atuais. Eles podem ser conferidos em https://dev.twitter.com/rest/public/rate-limits.

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=False)

O objeto `api` agora é nossa interface de acesso ao Twitter.

Podemos usá-la para saber quais são os trending topics nos diversos lugares onde essa informação é monitorada.

In [17]:
api.trends_available()

[{'country': '',
  'countryCode': None,
  'name': 'Worldwide',
  'parentid': 0,
  'placeType': {'code': 19, 'name': 'Supername'},
  'url': 'http://where.yahooapis.com/v1/place/1',
  'woeid': 1},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Winnipeg',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/2972',
  'woeid': 2972},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Ottawa',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3369',
  'woeid': 3369},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Quebec',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3444',
  'woeid': 3444},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Montreal',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3534',

In [90]:
trendtopics = api.trends_place(455824)
trendtopics

[{'as_of': '2017-03-30T12:15:39Z',
  'created_at': '2017-03-30T12:09:50Z',
  'locations': [{'name': 'Recife', 'woeid': 455824}],
  'trends': [{'name': '#PurposeTourRioDeJaneiro',
    'promoted_content': None,
    'query': '%23PurposeTourRioDeJaneiro',
    'tweet_volume': 911577,
    'url': 'http://twitter.com/search?q=%23PurposeTourRioDeJaneiro'},
   {'name': '#RedeBBB',
    'promoted_content': None,
    'query': '%23RedeBBB',
    'tweet_volume': 20926,
    'url': 'http://twitter.com/search?q=%23RedeBBB'},
   {'name': '#QuintaDetremuraSDV',
    'promoted_content': None,
    'query': '%23QuintaDetremuraSDV',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23QuintaDetremuraSDV'},
   {'name': '#New1DAlbum',
    'promoted_content': None,
    'query': '%23New1DAlbum',
    'tweet_volume': 30867,
    'url': 'http://twitter.com/search?q=%23New1DAlbum'},
   {'name': '#ParabensAnitta',
    'promoted_content': None,
    'query': '%23ParabensAnitta',
    'tweet_volume': 12907,
 

Como pode ser visto, o resultado é uma lista de objetos no formato JSON (um dicionário). Podemos usar Pandas para criar um data frame com os dados obtidos. Repare que a requisição retorna um único objeto com quatro chaves: data da apuração, data da requisição, local, e os tópicos. Nossa intenção é criar um data frame com os dados dos tópicos. As outras informações são complementares e podem ser replicadas no data frame se necessário após a criação.

In [91]:
dfTrendTopics = pd.DataFrame(trendtopics[0]['trends'])

In [92]:
dfTrendTopics.head()

,name,promoted_content,query,tweet_volume,url
0,#PurposeTourRioDeJaneiro,None,%23PurposeTourRioDeJaneiro,911577.0,http://twitter.com/search?q=%23PurposeTourRioD...
1,#RedeBBB,None,%23RedeBBB,20926.0,http://twitter.com/search?q=%23RedeBBB
2,#QuintaDetremuraSDV,None,%23QuintaDetremuraSDV,NaN,http://twitter.com/search?q=%23QuintaDetremuraSDV
3,#New1DAlbum,None,%23New1DAlbum,30867.0,http://twitter.com/search?q=%23New1DAlbum
4,#ParabensAnitta,None,%23ParabensAnitta,12907.0,http://twitter.com/search?q=%23ParabensAnitta


In [93]:
dfTrendTopics.sort_values(by='tweet_volume',ascending=False).\
                head(10)[['name','tweet_volume']]

,name,tweet_volume
0,#PurposeTourRioDeJaneiro,911577.0
23,Amazon,623836.0
15,Galaxy S8,363114.0
12,Justin Bieber,312288.0
24,Stories,259826.0
35,#BiTwitter,172114.0
30,#NoMoreSadSongsVideo,143841.0
13,Whindersson,118941.0
14,Neymar,98756.0
32,#Armys23Version,78788.0


Podemos também obter a lista de tweets de um usuário

In [43]:
trumpsTweets = api.user_timeline(screen_name='realdonaldtrump')
trumpsTweets[0]

Status(_json={'favorited': False, 'geo': None, 'is_quote_status': False, 'coordinates': None, 'id_str': '847061031293779969', 'retweet_count': 7674, 'in_reply_to_user_id_str': None, 'in_reply_to_status_id_str': None, 'entities': {'urls': [], 'hashtags': [], 'symbols': [], 'user_mentions': []}, 'retweeted': False, 'contributors': None, 'text': 'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!', 'created_at': 'Wed Mar 29 12:21:02 +0000 2017', 'in_reply_to_status_id': None, 'place': None, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_screen_name': None, 'in_reply_to_user_id': None, 'favorite_count': 35385, 'user': {'has_extended_profile': False, 'notifications': False, 'location': 'Washington, DC', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1980294624/DJT_Headshot_V2_normal.jpg', 'profile_sidebar_fill_color': 'C5CEC0', 'id_str'

O resultado da chamada é uma lista de objetos da classe `Status`. Inspecionando esse objeto, vemos que ele possui um atributo `_json` que armazena o tweet como um dicionário (JSON).

In [46]:
print(type(trumpsTweets[0]._json))
trumpsTweets[0]._json

<class 'dict'>


{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Mar 29 12:21:02 +0000 2017',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 35385,
 'favorited': False,
 'geo': None,
 'id': 847061031293779969,
 'id_str': '847061031293779969',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 7674,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': 'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Wed Mar 18 13:46:38 +0000 2009',
  'default_profile': False,
  'default_profile_image': False,
  'description': '45th Pre

Essa representação reflete o modelo de objetos estabelecido pela API do Twitter. Um tweet possui vários campos, como descrito em https://dev.twitter.com/overview/api/tweets. Nem todos os campos são interessantes para a análise de dados. Por exemplo, o tweet contém informações detalhadas sobre o usuário, como o link para a imagem de seu perfil, o link para a imagem que ele usa de fundo, entre outras coisas. Geralmente, esses dados não são úteis para análise. Assim, devemos ser cuidadosos para criar um data frame somente com os dados que julgarmos importantes.

Por exemplo, podemos selecionar os seguintes campos:

- id
- created_at
- entities (hashtags, urls, user_mentions)
- favorite_count
- in_reply_to_screen_name
- in_reply_to_status_id
- in_reply_to_user_id
- lang
- place (country, name)
- retweet_count
- source
- text
- truncated
- user (screen_name, id)

In [8]:
def filtra_campos(tweet):    
    dic = {a[0] : a[1] for a in tweet.items() if a[0] in ['id',
                                    'created_at',
                                    'favorite_count',
                                    'in_reply_to_screen_name',
                                    'in_reply_to_status_id',
                                    'in_reply_to_user_id',
                                    'lang',
                                    'retweet_count',
                                    'source',
                                    'text',
                                    'truncated']}
    dic.update(tweet['entities'])
    if tweet['place']:
        dic.update({'place_'+a[0] : a[1] for a in tweet['place'].items() \
                    if a[0] in ['country', 'name']})
    dic.update({'user_'+a[0] : a[1] for a in tweet['user'].items() \
                if a[0] in ['screen_name', 'id']})
    return dic
    

In [68]:
filtra_campos(trumpsTweets[0]._json)

{'created_at': 'Wed Mar 29 12:21:02 +0000 2017',
 'favorite_count': 35385,
 'hashtags': [],
 'id': 847061031293779969,
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'lang': 'en',
 'retweet_count': 7674,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'symbols': [],
 'text': 'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!',
 'truncated': False,
 'urls': [],
 'user_id': 25073877,
 'user_mentions': [],
 'user_screen_name': 'realDonaldTrump'}

Podemos aplicar a função `filtra_campos` aos tweets recuperados para extrair os campos desejados.

In [70]:
dfTrumpsTweets = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),trumpsTweets)))
dfTrumpsTweets.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Wed Mar 29 12:21:02 +0000 2017,35385,[],847061031293779969,None,None,None,en,NaN,7674,"<a href=""http://twitter.com/download/iphone"" r...",[],If the people of our great country could only ...,False,[],25073877,[],realDonaldTrump
1,Wed Mar 29 12:01:52 +0000 2017,30505,[],847056211006631936,None,None,None,en,NaN,6779,"<a href=""http://twitter.com/download/iphone"" r...",[],Remember when the failing @nytimes apologized ...,False,[],25073877,"[{'id_str': '807095', 'id': 807095, 'indices':...",realDonaldTrump
2,Tue Mar 28 22:41:09 +0000 2017,64865,[],846854703183020032,None,None,None,en,NaN,19235,"<a href=""http://twitter.com/download/iphone"" r...",[],Why doesn't Fake News talk about Podesta ties ...,False,[],25073877,"[{'id_str': '1367531', 'id': 1367531, 'indices...",realDonaldTrump
3,Tue Mar 28 21:48:40 +0000 2017,34999,"[{'indices': [31, 44], 'text': 'MadeInTheUSA'}]",846841493952319489,None,None,None,en,"[{'indices': [83, 106], 'media_url_https': 'ht...",7521,"<a href=""http://twitter.com/download/iphone"" r...",[],A NEW ERA IN AMERICAN ENERGY! \n#MadeInTheUSA🇺...,False,"[{'indices': [59, 82], 'url': 'https://t.co/EG...",25073877,[],realDonaldTrump
4,Tue Mar 28 20:57:17 +0000 2017,30731,[],846828561491202048,None,None,None,en,NaN,5840,"<a href=""http://twitter.com/download/iphone"" r...",[],It was an honor to welcome @GLFOP to the @Whit...,True,"[{'indices': [121, 144], 'url': 'https://t.co/...",25073877,"[{'id_str': '222196210', 'id': 222196210, 'ind...",realDonaldTrump


#### Pesquisando tweets

A API do Twitter fornece ainda acesso à pesquisa de tweets usando termos ou *hashtags*.
A descrição completa de como elaborar termos de busca pode ser encontrada em https://dev.twitter.com/rest/public/search. O acesso é fornecido em Tweepy pelo método [`search`](http://docs.tweepy.org/en/v3.5.0/api.html#API.search) da classe API.

A documentação de Tweepy traz a seguinte descrição

```python
API.search(q[, lang][, locale][, rpp][, page][, since_id][, geocode][, show_user])
```

Contudo, os parâmetros descritos estão desatualizados. A documentação do código traz a lista atual de parâmetros, conforme descrição da API do Twitter. São eles:

```python
'q', 'lang', 'locale', 'since_id', 'geocode',
             'max_id', 'since', 'until', 'result_type', 'count',
              'include_entities', 'from', 'to', 'source'
```

Verifique https://dev.twitter.com/rest/reference/get/search/tweets para ver a descrição de cada um dos parâmetros.


In [4]:
tweetsDataScience = api.search('#datascience',count=100,result_type="recent")
len(tweetsDataScience)

100

In [5]:
tweetsDataScience

[Status(entities={'user_mentions': [{'screen_name': 'AusDATAcentre', 'name': 'DataCentreTech', 'id_str': '830060806666670081', 'id': 830060806666670081, 'indices': [3, 17]}], 'hashtags': [{'indices': [41, 64], 'text': 'ArtificialIntelligence'}, {'indices': [66, 69], 'text': 'ai'}, {'indices': [71, 87], 'text': 'machinelearning'}, {'indices': [89, 97], 'text': 'bigdata'}, {'indices': [99, 103], 'text': 'iot'}, {'indices': [105, 118], 'text': 'deeplearning'}, {'indices': [120, 132], 'text': 'datascience'}], 'symbols': [], 'urls': []}, text='RT @AusDATAcentre: CONPUTER OR COMPUTER?\n#ArtificialIntelligence \n#ai \n#machinelearning \n#bigdata \n#iot \n#deeplearning \n#datascience \n#robo…', retweeted_status=Status(entities={'user_mentions': [], 'hashtags': [{'indices': [22, 45], 'text': 'ArtificialIntelligence'}, {'indices': [47, 50], 'text': 'ai'}, {'indices': [52, 68], 'text': 'machinelearning'}, {'indices': [70, 78], 'text': 'bigdata'}, {'indices': [80, 84], 'text': 'iot'}, {'indices': 

O resultado novamente é uma lista de objetos do tipo `Status`, a qual podemos processar da mesma forma como fizemos com os tweets do Trump.

In [9]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),
                                            tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,place_country,place_name,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Mon Apr 03 22:50:06 +0000 2017,0,"[{'indices': [41, 64], 'text': 'ArtificialInte...",849031280889020416,None,NaN,NaN,en,NaN,NaN,NaN,5,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @AusDATAcentre: CONPUTER OR COMPUTER?\n#Art...,False,[],68494106,"[{'screen_name': 'AusDATAcentre', 'name': 'Dat...",Pedra999
1,Mon Apr 03 22:49:49 +0000 2017,0,"[{'indices': [37, 49], 'text': 'DataScience'},...",849031208969412609,None,NaN,NaN,in,NaN,NaN,NaN,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],tp://RagingFX.com Report for Apr 2 #DataScie...,True,[{'display_url': 'twitter.com/i/web/status/8…'...,717581949280382977,[],AlertsRagingFX
2,Mon Apr 03 22:49:42 +0000 2017,0,"[{'indices': [75, 83], 'text': 'BigData'}, {'i...",849031181307830273,Storagepedia,8.489477e+17,2.192916e+09,en,NaN,NaN,NaN,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],@Storagepedia Thanks for sharing Paul! Great d...,False,[],14083628,"[{'screen_name': 'Storagepedia', 'name': 'Paul...",schmarzo
3,Mon Apr 03 22:49:19 +0000 2017,0,"[{'indices': [50, 53], 'text': 'ai'}, {'indice...",849031084260048898,None,NaN,NaN,en,NaN,NaN,NaN,4,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],"RT @ShaunVosper: I, ROBOT\nArtificialIntellige...",False,[],68494106,"[{'screen_name': 'ShaunVosper', 'name': 'Shaun...",Pedra999
4,Mon Apr 03 22:49:13 +0000 2017,0,"[{'indices': [89, 97], 'text': 'BigData'}, {'i...",849031061397012480,None,NaN,NaN,en,NaN,NaN,NaN,1,"<a href=""https://twitter.com/alevergara78"" rel...",[],RT @ygaudry: RT gp_pulipaka: Blockchain Market...,False,[],104520454,"[{'screen_name': 'ygaudry', 'name': 'yoann', '...",alevergara78


Agora podemos iniciar nosso processamento. Podemos, por exemplo, identificar as mensagens mais interessantes, considerando o número de retweets.

In [10]:
dfTweetsDataScience.retweet_count.describe()

count    100.000000
mean      14.860000
std       28.489594
min        0.000000
25%        1.000000
50%        3.500000
75%       16.000000
max      179.000000
Name: retweet_count, dtype: float64

In [11]:
[a for a in dfTweetsDataScience[\
    dfTweetsDataScience.retweet_count >= 16].sort_values(by='retweet_count')['text']]

['RT @HopwoodMedia: 6 key requirements for #IoT success via @JimMarous @evankirstel #analytics #bigdata #dataviz #DataScience https://t.co/bi…',
 'RT @HopwoodMedia: 6 key requirements for #IoT success via @JimMarous @evankirstel #analytics #bigdata #dataviz #DataScience https://t.co/bi…',
 'RT @gp_pulipaka: Correcting Intel’s Deep Learning Benchmark Mistakes. #BigData #DeepLearning #MachineLearning #DataScience #AI\nhttps://t.co…',
 'RT @MrDataScience: Apollo 17 was the last time humans travelled beyond low Earth orbit (45 years ago?!?!) #SpaceX #DataScience https://t.co…',
 'RT @KirkDBorne: Predicting Customer Behavior with Predictive #BehavioralAnalytics [#infographic]\n\n#BigData #DataScience #MachineLearning #C…',
 'RT @gp_pulipaka: Application of K-Means in #DataMining to Cluster Hemodialysis Patients. #BigData #MachineLearning #DataScience #AI \nhttps:…',
 'RT @gp_pulipaka: Stanford researchers create #DeepLearning algorithm for drug development. #BigData #MachineLearning #DataSc

Observamos que muitas mensagens se tratam de retweets extra-oficiais (não executadas pelo botão de retweet). Essas mensagens citam diversas usuários. Dessa forma, podemos os tweets para encontrar quais são os usuários mais influentes em data science.

Se verificarmos a descrição do objeto Tweet, veremos que ele possui um atributo user_mentions no campo ***entities***. Esse atributo é preenchido pelo Twitter que entrega uma lista de todos os usuários (objetos do tipo User) contidos no texto. Para o nosso exemplo, podemos nos restringir somente ao login do usuário no perfil (screen_name). Depois, podemos usar o screen_name dos usuários mais influentes para recuperar as informações de seus perfis diretamente da API. 

In [12]:
usuariosInfluentes = dfTweetsDataScience.user_mentions.apply(lambda x: 
                                        [y['screen_name'] for y in x])
usuariosInfluentes

0                                       [AusDATAcentre]
1                                                    []
2                                        [Storagepedia]
3                                         [ShaunVosper]
4                                             [ygaudry]
5                                                    []
6                                         [ShaunVosper]
7                                         [gp_pulipaka]
8                                                    []
9                                         [ShaunVosper]
10                                      [AusDATAcentre]
11                                        [gp_pulipaka]
12                                            [msarsar]
13                                        [ShaunVosper]
14                                        [gp_pulipaka]
15                                        [gp_pulipaka]
16                                      [AusDATAcentre]
17                                            [p

In [13]:
top10Influentes = Counter(reduce(lambda x, y: x+y,
                                 usuariosInfluentes.values)).most_common(10)
top10Influentes

[('KirkDBorne', 7),
 ('ShaunVosper', 6),
 ('gp_pulipaka', 6),
 ('ipfconline1', 6),
 ('AusDATAcentre', 5),
 ('CouthonConseil', 4),
 ('essec', 4),
 ('bigdataconf', 4),
 ('MikeTamir', 4),
 ('msarsar', 3)]

Agora que encontramos os dez usuários mais citados nas mensagens, podemos usar a API para recuperar seus dados de perfil. Essa consulta é feita pela interface `lookup_users`.

In [14]:
descTop10Influentes = api.lookup_users(screen_names=[a[0] for a in top10Influentes])
descTop10Influentes

[User(profile_image_url_https='https://pbs.twimg.com/profile_images/834247069057613825/spqkv7Ko_normal.jpg', screen_name='KirkDBorne', profile_text_color='333333', profile_background_image_url_https='https://pbs.twimg.com/profile_background_images/378800000110442114/094a0b4d38b70e3871a1bda7d7fe2341.jpeg', id_str='534563976', profile_link_color='0084B4', profile_sidebar_fill_color='DDEEF6', notifications=False, profile_sidebar_border_color='FFFFFF', translator_type='none', name='Kirk Borne', profile_banner_url='https://pbs.twimg.com/profile_banners/534563976/1450104430', is_translation_enabled=False, lang='en', _api=<tweepy.api.API object at 0x11117dcc0>, id=534563976, followers_count=141937, follow_request_sent=False, location='Booz Allen Hamilton', status=Status(entities={'user_mentions': [{'screen_name': 'benhamner', 'name': 'Ben Hamner', 'id_str': '22674817', 'id': 22674817, 'indices': [15, 25]}, {'screen_name': 'Quora', 'name': 'Quora', 'id_str': '33696409', 'id': 33696409, 'indice

Os objetos do tipo User possuem diversos campos que não são úteis para nossa análise nesse momento. Assim, antes de criar um data frame com os dados dos usuários, devemos filtrá-los, escolhendo somente aquilo que pode ser interessante. Nesse primeiro momento restringiremos nossa análise ao login, nome, descrição, localização, número de seguidores, o número de pessoas que o usuário segue, e o número de tweets que ele já tuitou.

In [15]:
dfTop10Influentes = pd.DataFrame([{'screen_name':a.screen_name,
                                  'name':a.name,
                                   'description':a.description,
                                   'location':a.location,
                                   'followers_count':a.followers_count,
                                   'friends_count':a.friends_count,
                                   'no_tweets':a.statuses_count
                                  } for a in descTop10Influentes],
                                columns = ['screen_name', 'name', 'description',
                                          'location', 'followers_count', 'friends_count', 
                                           'no_tweets'])
dfTop10Influentes

,screen_name,name,description,location,followers_count,friends_count,no_tweets
0,KirkDBorne,Kirk Borne,"The Principal Data Scientist at @BoozAllen, Ph...",Booz Allen Hamilton,141937,81196,69321
1,ShaunVosper,ShaunVosper,Data technology specialist & father of beautif...,"Brisbane, Queensland",286,945,450
2,gp_pulipaka,Dr. GP Pulipaka,Ganapathi Pulipaka | Founder and CEO @deepsing...,"Los Angeles, CA",21148,12649,27043
3,ipfconline1,ipfconline,Création de Sites Internet Formation et Consei...,"Marseille, France",38594,38138,53125
4,AusDATAcentre,DataCentreTech,Australia's premier Data Centre Experts. We'll...,"Brisbane, Queensland",1077,4468,504
5,CouthonConseil,✦ Couthon Conseil ✦,#Recrutement de #cadres & #freelances en {#Big...,France & Worldwide,6239,5220,7430
6,essec,ESSEC BusinessSchool,Leading International Business School. Excelle...,Paris - Singapore - Rabat,28322,2756,26349
7,bigdataconf,Big Data Conference,Hurry!! Register fast #BigData Bootcamp #Dalla...,,4853,2439,14692
8,MikeTamir,"Mike Tamir, PhD",#CSO Chief #DataScientist tweeting on #DataSci...,"San Francisco, USA",12290,4751,2976
9,msarsar,Dr. Mehdi Sarsar,ICT Senior Consultant 🇬🇧🇫🇷 #BigData #Economics...,Paris - London,7685,2959,3212


#### Trabalhando com cursores

A API do Twitter fornece mecanismos para que o usuário itere sobre o conjunto de resultados de uma requisição. Ao invés de retornar todos os resultados de uma única vez, a API utiliza o conceito de paginação através de cursores. 

Os resultados são divididos em páginas. Na primeira requisição, assume que a página solicitada é a primeira, usando para isso o parâmetro cursor com valor -1. Junto com os resultados, a API retorna o cursor (índice de página) anterior e o próximo. Na requisição subsequente, o usuário informa o próximo cursor e a API retorna a página seguinte com os outros resultados. O usuário pode, então, iterar sobre mais resultados que o limite imposto em cada requisição.

A biblioteca Tweepy possui uma interface direta para a paginação usada pelo Twitter. O tutorial http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html detalha como utilizar os cursores. Veremos um exemplo a seguir onde recuperaremos os 1000 tweets mais recentes contendo #datascience.

In [91]:
tweetsDataScience = [tweet for tweet in tweepy.Cursor(api.search, q='#datascience',\
                                  count=100,result_type="recent").items(1000)]
len(tweetsDataScience)

1000

O resultado da pesquisa continua sendo uma lista de objetos do tipo `Status`, os quais podemos processar da mesma forma que fizemos com a consulta direta.

In [92]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),
                                            tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,place_country,place_name,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Mon Apr 03 16:37:52 +0000 2017,0,"[{'indices': [76, 84], 'text': 'BigData'}, {'i...",848937606800957443,None,NaN,NaN,en,NaN,NaN,NaN,4,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @gp_pulipaka: Understanding Neural-Networks...,False,[],480875170,"[{'name': 'Dr. GP Pulipaka', 'indices': [3, 15...",Rosenchild
1,Mon Apr 03 16:37:44 +0000 2017,0,"[{'indices': [70, 78], 'text': 'BigData'}, {'i...",848937574160904193,None,NaN,NaN,en,NaN,NaN,NaN,6,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @gp_pulipaka: Neural-Networks: Part III – D...,False,[],480875170,"[{'name': 'Dr. GP Pulipaka', 'indices': [3, 15...",Rosenchild
2,Mon Apr 03 16:37:33 +0000 2017,0,"[{'indices': [60, 68], 'text': 'BigData'}, {'i...",848937527759314945,None,NaN,NaN,en,NaN,NaN,NaN,4,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @gp_pulipaka: A.I. used to create worldwide...,False,"[{'display_url': 'buff.ly/2otsdDu', 'indices':...",480875170,"[{'name': 'Dr. GP Pulipaka', 'indices': [3, 15...",Rosenchild
3,Mon Apr 03 16:37:27 +0000 2017,0,"[{'indices': [70, 78], 'text': 'BigData'}, {'i...",848937500064264192,None,NaN,NaN,en,NaN,NaN,NaN,6,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @gp_pulipaka: Correcting Intel’s Deep Learn...,False,[],480875170,"[{'name': 'Dr. GP Pulipaka', 'indices': [3, 15...",Rosenchild
4,Mon Apr 03 16:37:16 +0000 2017,0,"[{'indices': [45, 58], 'text': 'DeepLearning'}...",848937454254116864,None,NaN,NaN,en,NaN,NaN,NaN,7,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @gp_pulipaka: Stanford researchers create #...,False,[],480875170,"[{'name': 'Dr. GP Pulipaka', 'indices': [3, 15...",Rosenchild


In [96]:
dfTweetsDataScience[['favorite_count','retweet_count']].describe()

,favorite_count,retweet_count
count,1000.000000,1000.000000
mean,0.446000,11.303000
std,1.632712,22.274904
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,4.000000
75%,0.000000,12.000000
max,21.000000,178.000000


In [93]:
top10Influentes = Counter(reduce(lambda x, y: x+y,
                                 dfTweetsDataScience.user_mentions.apply(lambda x: 
                                        [y['screen_name'] for y in x]).values)).most_common(10)
top10Influentes

[('gp_pulipaka', 97),
 ('KirkDBorne', 70),
 ('ipfconline1', 44),
 ('DataScienceCtrl', 38),
 ('evankirstel', 33),
 ('miguelselas', 30),
 ('Ronald_vanLoon', 25),
 ('kdnuggets', 24),
 ('DBaker007', 20),
 ('MikeQuindazzi', 18)]

In [97]:
dfTop10Influentes = pd.DataFrame([{'screen_name':a.screen_name,
                                  'name':a.name,
                                   'description':a.description,
                                   'location':a.location,
                                   'followers_count':a.followers_count,
                                   'friends_count':a.friends_count,
                                   'no_tweets':a.statuses_count
                                  } \
                                  for a in api.lookup_users(screen_names=[a[0] \
                                                                          for a in top10Influentes])],
                                columns = ['screen_name', 'name', 'description',
                                          'location', 'followers_count', 'friends_count', 
                                           'no_tweets'])
dfTop10Influentes

,screen_name,name,description,location,followers_count,friends_count,no_tweets
0,gp_pulipaka,Dr. GP Pulipaka,Ganapathi Pulipaka | Founder and CEO @deepsing...,"Los Angeles, CA",21126,12522,27043
1,KirkDBorne,Kirk Borne,"The Principal Data Scientist at @BoozAllen, Ph...",Booz Allen Hamilton,141908,81199,69311
2,ipfconline1,ipfconline,Création de Sites Internet Formation et Consei...,"Marseille, France",38551,38054,53057
3,DataScienceCtrl,Data Science Central,Co-founded by Vincent Granville and part the D...,"Los Angeles, CA",87166,1126,18838
4,evankirstel,Evan Kirstel,#B2B #Solopreneur #Influencer #ThoughtLeader H...,"Boston, MA",111743,96540,475684
5,miguelselas,Miguel Selas,Managing Partner & CEO at Bufete de Marketing ...,Madrid | México DF | São Paulo,6435,5405,5056
6,Ronald_vanLoon,Ronald van Loon,Helping data driven companies generating value...,#NL. Also connect on LinkedIn,78336,77253,24948
7,kdnuggets,KDnuggets,"Covering #Analytics, #BigData, #DataMining, #D...","Brookline, MA, USA",79581,419,36923
8,DBaker007,Duane Baker,IT Consultant #IT #bigdata #analytics #cloud #...,"Columbus, OH",23073,22013,55907
9,MikeQuindazzi,Mike Quindazzi ✨,Managing Director • @StrategyAnd @PwC • Execut...,"California, USA",29193,4668,18090


## Usando a API de Streaming

Como foi dito anteriormente, a API REST não foi desenvolvida para a coleta de dados em tempo real. De acordo com a documentação da API (https://dev.twitter.com/rest/reference/get/search/tweets), o mecanismo de busca não foi desenvolvido para servir como base para coleta exaustiva de tweets. Veja o que diz a documentação

    Please note that Twitter’s search service and, by extension, the Search API is not meant to be an exhaustive source of Tweets. Not all Tweets will be indexed or made available via the search interface.
    
Caso seja importante coletar um grande volume de dados para processá-los depois, deve-se usar a [interface/API de streaming](https://dev.twitter.com/streaming/overview). Nesse caso, uma conexão é mantida com o servidor do Twitter por um tempo mais longo. Novos tweets serão enviados ao usuário à medida em que são postados.

Mais uma vez, Tweepy fornece os mecanismos necessários para coletas da API de streaming. Em suma, o usuário deve criar um *listener* que executará as ações correspondentes quando um evento ocorrer. Esse listerner é passado para Tweepy que se encarrega de conectar com a API do Twitter.

In [99]:
from tweepy.streaming import StreamListener
from tweepy import Stream

A classe abaixo define um exemplo de um listener que exibe as mensagens na saída padrão sempre que um novo dado é enviado.

In [100]:
class StdOutListener(StreamListener):

    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)


O segundo passo para se coletar dados da API de streaming é instanciar um objeto `Stream` que será responsável pela conexão. Devemos passar para o construtor a sessão autenticada da mesma forma que na API REST, e um listener.

In [101]:
stream = Stream(auth, StdOutListener())

Na maior parte dos casos, nossa intenção é em coletar apenas tweets específicos de um assunto. Assim como na interface de busca, podemos informar os termos que estamos interessados no parâmetro **track**. Uma descrição mais completa dos parâmetros está disponível em https://dev.twitter.com/streaming/overview/request-parameters

In [ ]:
stream.filter(track=['vamosport' ,'vamosanta', 
                     'classicodasmultidoes', 
                     'sportxsantacruz', 'sport recife'])